In [ ]:
from models.helper import GetModel
from dataloaders.train_loader import GSVCitiesDataset
from dataloaders.test_loader import TestDataset
from torch.utils.data import DataLoader
import torch
import os
from pytorch_metric_learning import losses, miners
from train import train_model
from evaluation import eval_model
from utils.lr_scheduler import custom_scheduler
import random
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)


train_dataset = GSVCitiesDataset(generated_data_prob=0)
test_dataset = TestDataset()

num_workers = 16
train_dataloader = DataLoader(train_dataset, 
                              batch_size=100, 
                              shuffle=True, 
                              num_workers=num_workers, 
                              pin_memory=True
                              )

test_dataloader = DataLoader(test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
sf_xs_test_dataset = TestDataset(path='./data/sf_xs/test')
sf_xs_test_dataloader = DataLoader(sf_xs_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
tokyo_test_dataset = TestDataset(path='./data/tokyo_xs/test')
tokyo_test_dataloader = DataLoader(tokyo_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )




# Base Model

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_custom'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

# MixVPR

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_mixvpr_custom'

model = GetModel(aggregator='mixvpr', input_size=256, output_size=512)
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'patience': 45,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

# Depth Models

In [ ]:
from models.helper import GetModel, GetMultiModel, GetDeAttNet
from dataloaders.train_loader import GSVCitiesDataset
from dataloaders.test_loader import TestDataset
from torch.utils.data import DataLoader
import torch
import os
from pytorch_metric_learning import losses, miners
from train import train_model
from evaluation import eval_model
from utils.lr_scheduler import custom_scheduler
import random
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)


train_dataset = GSVCitiesDataset(generated_data_prob=0, multi_model=True)
test_dataset = TestDataset(multi_model=True)

num_workers = 16
train_dataloader = DataLoader(train_dataset, 
                              batch_size=100, 
                              shuffle=True, 
                              num_workers=num_workers, 
                              pin_memory=True
                              )

test_dataloader = DataLoader(test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
sf_xs_test_dataset = TestDataset(path='./data/sf_xs/test', multi_model=True)
sf_xs_test_dataloader = DataLoader(sf_xs_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
tokyo_test_dataset = TestDataset(path='./data/tokyo_xs/test', multi_model=True)
tokyo_test_dataloader = DataLoader(tokyo_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )


torch.cuda.empty_cache()

# DEPTH MIXVPR

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_mixvpr_arda_512'

model = GetMultiModel(aggregator='mixvpr', use_fusion=False, input_size=512, output_size=512)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

# DeAttNet

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_deattnet'

model = GetDeAttNet(aggregator='gem')
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()